In [19]:
from pathlib import Path
import pandas as pd
from bitstruct import unpack_from as upf

In [15]:
DIR = Path(r"C:\Users\ucasbwh\Desktop\bin")

HKfiles = sorted(DIR.rglob("*HK.txt"))
print(HKfiles)

[WindowsPath('C:/Users/ucasbwh/Desktop/bin/ERR_INJ_1_FWL_HK.txt'), WindowsPath('C:/Users/ucasbwh/Desktop/bin/ERR_INJ_2_WACR_CMD_LOST_IAO_STUCK_HK.txt'), WindowsPath('C:/Users/ucasbwh/Desktop/bin/ERR_INJ_3_HRC_MM_STUCK_HK.txt'), WindowsPath('C:/Users/ucasbwh/Desktop/bin/ERR_INJ_4_BUILD_IN_ERRORS_HK.txt'), WindowsPath('C:/Users/ucasbwh/Desktop/bin/PANCAM_HRC_Calibration_HK.txt'), WindowsPath('C:/Users/ucasbwh/Desktop/bin/PANCAM_WAC_RR_HK.txt')]


In [96]:
reading.stem

'PANCAM_WAC_RR_HK'

In [6]:
reading = HKfiles[-1]
log_data = open(reading, 'r')
#columns = ['Text', 'TS_Byte', 'RAW']
DT = pd.read_table(reading, sep=']', header=None)
DT.head()

,0,1
0,Timestamp: 1554829292559[Informative,0x21 0x01 0x0C 0x00 0x41 0x00 0x03 0x00 0x00 0...
1,Timestamp: 1554829293129[Informative,0x21 0x01 0x0C 0x00 0x41 0x00 0x08 0x00 0x00 0...
2,Timestamp: 1554829294159[Informative,0x21 0x01 0x0C 0x00 0x41 0x00 0x11 0x00 0x00 0...
3,Timestamp: 1554829295216[Informative,0x21 0x01 0x0C 0x00 0x41 0x00 0x1A 0x00 0x00 0...
4,Timestamp: 1554829296239[Informative,0x21 0x01 0x0C 0x00 0x41 0x00 0x23 0x00 0x00 0...


In [51]:
DL = pd.DataFrame()
DL['SPW_RAW'] = DT[1].apply(lambda x: x.replace('0x','').replace(' ',''))
DL.head()

,SPW_RAW
0,21010C0041000300000058FF150700000504000000004D...
1,21010C004100080000005849150700000505000000004D...
2,21010C0041001100000048683503000006BF03D700003D...
3,21010C0041001A00000058C21507000006C003D700004D...
4,21010C00410023000000580E1507000006C103D700004D...


In [76]:
DL['HK_RAW'] = DL.SPW_RAW.apply(lambda x: x[108-84:-2])

In [60]:
DT['Unix'] = DT[0].apply(lambda x: x[11:-12])

In [77]:
DL['DT'] = pd.to_datetime(DT['Unix'], unit='ms')
DL.head()

,SPW_RAW,HK_RAW,DT
0,21010C0041000300000058FF150700000504000000004D...,150700000504000000004D0007090484073B06B806B806...,2019-04-09 17:01:32.559
1,21010C004100080000005849150700000505000000004D...,150700000505000000004D0007090484073B06B806B806...,2019-04-09 17:01:33.129
2,21010C0041001100000048683503000006BF03D700003D...,3503000006BF03D700003D0007090484073B06B806B806...,2019-04-09 17:01:34.159
3,21010C0041001A00000058C21507000006C003D700004D...,1507000006C003D700004D0007090484073B06B806B806...,2019-04-09 17:01:35.216
4,21010C00410023000000580E1507000006C103D700004D...,1507000006C103D700004D0007090484073B06B806B806...,2019-04-09 17:01:36.239


In [84]:
Bin = DL['HK_RAW'].apply(lambda x: bytearray.fromhex(x))
Bin.head()

0    [21, 7, 0, 0, 5, 4, 0, 0, 0, 0, 77, 0, 7, 9, 4...
1    [21, 7, 0, 0, 5, 5, 0, 0, 0, 0, 77, 0, 7, 9, 4...
2    [53, 3, 0, 0, 6, 191, 3, 215, 0, 0, 61, 0, 7, ...
3    [21, 7, 0, 0, 6, 192, 3, 215, 0, 0, 77, 0, 7, ...
4    [21, 7, 0, 0, 6, 193, 3, 215, 0, 0, 77, 0, 7, ...
Name: HK_RAW, dtype: object

In [85]:
## Test 1 Block Type is always 0
Extract = Bin.apply(lambda x: upf('u1', x, offset=0)[0]).astype('int64')
if True in (Extract !=0).unique():
    print("Fail")

In [86]:
## Test 2 PanCam ID is always 5
Extract = Bin.apply(lambda x: upf('u4', x, offset=4)[0]).astype('int64')
if True in (Extract !=5).unique():
    print("Fail")

In [87]:
## Test 3 Reserved Byte is always 0
Extract = Bin.apply(lambda x: upf('u8', x, offset=11*8)[0]).astype('int64')
if True in (Extract !=0).unique():
    print("Fail")

In [102]:
DIR = Path(r"C:\Users\ucasbwh\Desktop\bin\PROC\PANCAM_WAC_RR_HK")

HKfiles = sorted(DIR.rglob("*HKTM*"))
print(HKfiles)

RTM = pd.read_pickle(HKfiles[0])

[WindowsPath('C:/Users/ucasbwh/Desktop/bin/PROC/PANCAM_WAC_RR_HK/PANCAM_WAC_RR_HK_Unproc_HKTM.pickle')]


In [103]:
RTM

,SPW_RAW,RAW,Unix,DT
0,21010C0041000300000058FF150700000504000000004D...,150700000504000000004D0007090484073B06B806B806...,1554829292559,2019-04-09 17:01:32.559
1,21010C004100080000005849150700000505000000004D...,150700000505000000004D0007090484073B06B806B806...,1554829293129,2019-04-09 17:01:33.129
2,21010C0041001100000048683503000006BF03D700003D...,3503000006BF03D700003D0007090484073B06B806B806...,1554829294159,2019-04-09 17:01:34.159
3,21010C0041001A00000058C21507000006C003D700004D...,1507000006C003D700004D0007090484073B06B806B806...,1554829295216,2019-04-09 17:01:35.216
4,21010C00410023000000580E1507000006C103D700004D...,1507000006C103D700004D0007090484073B06B806B806...,1554829296239,2019-04-09 17:01:36.239
...,...,...,...,...
115,21010C004103F900000048E935030000073103D700003D...,35030000073103D700003D0007090484073B06B806B806...,1554829408981,2019-04-09 17:03:28.981
116,21010C0041040000000058D915070000073203D700004D...,15070000073203D700004D0007090484073B06B806B806...,1554829409810,2019-04-09 17:03:29.810
117,21010C00410409000000587C15070000073303D700004D...,15070000073303D700004D0007090484073B06B806B806...,1554829410849,2019-04-09 17:03:30.849
118,21010C00410412000000585215070000073403D700004D...,15070000073403D700004D0007090484073B06B806B806...,1554829411879,2019-04-09 17:03:31.879


In [104]:
Bin = RTM['RAW'].apply(lambda x: bytearray.fromhex(x))

In [107]:
def PandUPF(Column, Len, OffBy, OffBi):
    """Extracts a single RAW value from a binary pandas data column"""
    if int(Len[1:]) > 63:
        raise ValueError("PandUPF used for variable larger than 63 bits. Returned value is cast to an Int64")
    Extract = Column.apply(lambda x: upf(Len, x, offset=8*OffBy+OffBi)[0]).astype('Int64')
    return Extract

In [109]:
TM = pd.DataFrame()
TM['TM_Type_ID']     = PandUPF(Bin, 'u6', 0, 8)
TM

,TM_Type_ID
0,1
1,1
2,0
3,1
4,1
...,...
115,0
116,1
117,1
118,1


In [114]:
NEBin = Bin[TM['TM_Type_ID']==1]
display(NEBin)

0      [21, 7, 0, 0, 5, 4, 0, 0, 0, 0, 77, 0, 7, 9, 4...
1      [21, 7, 0, 0, 5, 5, 0, 0, 0, 0, 77, 0, 7, 9, 4...
3      [21, 7, 0, 0, 6, 192, 3, 215, 0, 0, 77, 0, 7, ...
4      [21, 7, 0, 0, 6, 193, 3, 215, 0, 0, 77, 0, 7, ...
5      [21, 7, 0, 0, 6, 194, 3, 215, 0, 0, 77, 0, 7, ...
                             ...                        
114    [21, 7, 0, 0, 7, 48, 3, 215, 0, 0, 77, 0, 7, 9...
116    [21, 7, 0, 0, 7, 50, 3, 215, 0, 0, 77, 0, 7, 9...
117    [21, 7, 0, 0, 7, 51, 3, 215, 0, 0, 77, 0, 7, 9...
118    [21, 7, 0, 0, 7, 52, 3, 215, 0, 0, 77, 0, 7, 9...
119    [21, 7, 0, 0, 7, 53, 3, 215, 0, 0, 77, 0, 7, 9...
Name: RAW, Length: 93, dtype: object

In [129]:
TM['TM_Type_ID'].head()

0    1
1    1
2    0
3    1
4    1
Name: TM_Type_ID, dtype: Int64

In [130]:
Bin.head()

0    [21, 7, 0, 0, 5, 4, 0, 0, 0, 0, 77, 0, 7, 9, 4...
1    [21, 7, 0, 0, 5, 5, 0, 0, 0, 0, 77, 0, 7, 9, 4...
2    [53, 3, 0, 0, 6, 191, 3, 215, 0, 0, 61, 0, 7, ...
3    [21, 7, 0, 0, 6, 192, 3, 215, 0, 0, 77, 0, 7, ...
4    [21, 7, 0, 0, 6, 193, 3, 215, 0, 0, 77, 0, 7, ...
Name: RAW, dtype: object

In [131]:
NEBin.head()

0    [21, 7, 0, 0, 5, 4, 0, 0, 0, 0, 77, 0, 7, 9, 4...
1    [21, 7, 0, 0, 5, 5, 0, 0, 0, 0, 77, 0, 7, 9, 4...
3    [21, 7, 0, 0, 6, 192, 3, 215, 0, 0, 77, 0, 7, ...
4    [21, 7, 0, 0, 6, 193, 3, 215, 0, 0, 77, 0, 7, ...
5    [21, 7, 0, 0, 6, 194, 3, 215, 0, 0, 77, 0, 7, ...
Name: RAW, dtype: object

In [133]:
NEBin[82:83]

108    [21, 7, 0, 0, 7, 43, 3, 215, 0, 0, 77, 0, 7, 9...
Name: RAW, dtype: object

In [93]:
## Test 4 Reserved Byte is always 0
Extract = Bin.apply(lambda x: upf('u16', x, offset=12*8)[0]).astype('int64')
if True in (Extract !=1801).unique():
    print("Fail")